## Understand NETCONF/YANG and build your first NETCONF client with Python to interwork with IOS-XR

This laboratory addresses the functionality of IOS XR to communicate over the Network Configuration Protocol (NETCONF). This functionality benefits the remote configuration of IOS XR devices through configuration managers that act as NETCONF clients.

The current infrastructure of IOS XR benefits the demand for network automation of increasingly bigger networks. The infrastructure offers flexibility in the range of management protocols, as well as in encoding formats. The wide featureset of YANG models includes native, OpenConfig and IETF models and allows a programmable interaction with such network devices. Lastly, the decoupling of these three components within the infrastructure show its adaptability to preferences for networking and programming tools.

<img src="images/configuration management.png">

<mark>NOTE</mark>

References on the NETCONF protocol:
 - [Network Configuration Protocol (NETCONF) - RFC 6241](https://tools.ietf.org/html/rfc6241)
 - [NETCONF Event Notifications - RFC 5277](https://tools.ietf.org/html/rfc5277)

References on NETCONF-based programmability on IOS XR:


 ASR 9000
  - [Programmability Configuration Guide](https://www.cisco.com/c/en/us/td/docs/routers/asr9000/software/asr9k-r7-0/programmability/configuration/guide/b-programmability-cg-asr9000-70x/b-programmability-cg-asr9000-70x_chapter_010.html)
  - [Yang Data Models](https://www.cisco.com/c/en/us/td/docs/routers/asr9000/software/asr9k-r7-0/programmability/configuration/guide/b-programmability-cg-asr9000-70x/b-programmability-cg-asr9000-70x_chapter_01.html)
  - [Telemetry Configuration](https://www.cisco.com/c/en/us/td/docs/routers/asr9000/software/asr9k-r7-0/telemetry/configuration/guide/b-telemetry-cg-asr9000-70x.html)



 NCS 5000
  - [Programmability Configuration Guide](https://www.cisco.com/c/en/us/td/docs/iosxr/ncs5000/programmability/70x/b-programmability-cg-ncs5000-70x/b-programmability-cg-ncs5000-70x_chapter_010.html)
  - [YANG Data Models](https://www.cisco.com/c/en/us/td/docs/iosxr/ncs5000/programmability/70x/b-programmability-cg-ncs5000-70x/b-programmability-cg-ncs5000-70x_chapter_01.html)
  - [Telemetry Configuration](cisco.com/c/en/us/td/docs/iosxr/ncs5000/programmability/70x/b-programmability-cg-ncs5000-70x.html)

**NETCONF overview**

NETCONF is a communication protocol for network management, used over Secure Shell (SSH) transport. Network devices will typically run NETCONF servers and a NETCONF client / application will use Remote Procedure Calls (RPCs) to retrieve the state of a device or to modify a device's configuration. A NETCONF operation and its payload are encoded in the Extensible Markup Format (XML).

IOS XR comes with a NETCONF server that can be activated using the following commands:
 ```
 netconf-yang agent ssh
 netconf agent tty
 
 ssh server v2
 ssh server netconf vrf default
 ```
 
 By default, the server will open port 830 for NETCONF communication.
 
 **Configuration datastores in NETCONF**
 
 A NETCONF server will have defined one or more configuration datastores available for configuration management. Such a datastore contains configuration needed to transition the device from an initial state to the intended operational state. The `<running>` configuration datastore holds the current active configuration on the device and is present in the base implementation of the protocol. Support for more datastores can be defined by extending the capabilities of a device.
 
 In the case of IOS XR, there are two datastores available:
 - `<running>` - it holds the active configuration of the device
 - `<candidate>` - it holds configuration that can be changed without affecting the running configuration; the changes can be committed to the running configuration datastore or they can be discarded
 
 
 **Protocol operations in NETCONF**
 
NETCONF supports operations for configuration management and state retrieval. The operations from the base protocol are:
- **`get`** - the client requests a copy of the running configuration and of the device's state 
- **`get-config`** - the client requests a copy of the configuration of a datastore; filters are supported
- **`edit-config`** - the client asks the server to apply modifications to the configuration of a target datastore
- `copy-config` - the client asks the server to create or overwrite the entire target configuration datastore with the contents of another datastore
- `delete-config` - the client asks the server to delete the target configuration datastore
- **`lock`** - the client locks a server's target configuration datastore; this is intended for short time periods when changes are being applied and interference with other clients or management entities is unwanted 
- **`unlock`** - the client releases the lock from a server's target configuration datastore
- **`close-session`** - the client gracefully ends its NETCONF session with the server
- `kill-session` - the client forcely ends its NETCONF session with the server, aborting ongoing operations

In this laboratory you will practice the operations highlighted in bold.

### Connect over NETCONF to the IOS XR device

 <mark>TASK 1</mark>
 
 Load parameters for the connection with the IOS XR device:

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('workshop-configuration.ini')
device=config['DEVICE']

 In this laboratory we will use the `ncclient` library that is a Python implementation of functionalities for a NETCONF client.
 
 <mark>NOTE</mark> - For more information about the `ncclient` see:
 - https://github.com/ncclient/ncclient
 - https://developer.cisco.com/codeexchange/github/repo/ncclient/ncclient/
 
 
 <mark>TASK 2</mark>
 
 Connect to the IOS XR device:

In [ ]:
from ncclient import manager

netconf_connection = manager.connect(host=device['Host'], 
                                     port=device['NCPort'],
                                     username=device['UserName'],
                                     password=device['Password'],
                                     hostkey_verify=False,
                                     device_params={'name':'iosxr'})

### Retrieval of the capabilities advertised by the IOS XR device

During the session establishment, both the client and the server will verify that they have a common protocol version. Additionally, they will exchange the capabilities that are supported.

 <mark>TASK 3</mark>
 
 Retrieve the capabilities of IOS XR. Notice the first 7 capabilities with URI of the form `urn:ietf:params:netconf:...` - these are capabilities implemented as per the RFC 6241 and RFC 5277. The rest of the capabilities represent YANG data models and are explained in the next section of this laboratory.

In [ ]:
for capability in netconf_connection.server_capabilities:
    print (capability)

## Model-driven programmability

#### YANG - A Data Modelling Language

A data model is written in YANG data modelling language and is used for describing configuration data and operational states in a hierarchical manner. YANG data models are defined in highly structured text files and are compilabl. YANG's statically-typed syntax and semantic definition ensures the verification of the model's definition at compile time, resulting in errors that can be treated early in the development process.

YANG data models can be instantiated with an XML definition. This type of instantiation is powerful and binds the purpose of NETCONF to the abstractization of the device management: one can now configure the device over NETCONF, using XML instantiations of all the supported data models advertised by the device. This allows for automatization of network management operations, replacing the command line configuration that is prone to human error, with a programming-friendly structure.
 
 <mark>NOTE</mark> - Reference on YANG:
 - [RFC 6020 - YANG - A Data Modeling Language for the Network Configuration Protocol (NETCONF)](https://tools.ietf.org/html/rfc6020)
 - [The YANG 1.1 Data Modeling Language - RFC 7950](https://tools.ietf.org/html/rfc7950)
 
 
 Throughout the rest of this laboratory we will explore the NETCONF operations for automated configuration of the IOS XR device and monitor the data rate of generated traffic. The configuration and data streaming will be based on YANG models, i.e., model-driven:
 
<img src="images/model-driven programmability.png">

### Retrieval of the configuration for model-driven telemetry streaming (dial-out)

The IOS XR device has been preconfigured for model-driven telemetry (MDT) streaming using the gRPC dial-out mechanism.

In this case, the device initiates a gRPC session (dial-out) with the MDT receiver (i.e., the host that runs the Jupyter notebook - 198.18.134.50) and exchanges SYN -- SYN-ACK -- ACK with it while establishing the connection.

If the connection is successfully created, the device will start streaming telemetry data towards the collection point.

The configuration consists of:

- A **sensor-group** that is composed of sensor-path(s):
  - A **sensor-path** is a specification of the YANG component about which information should be streamed. It is specified in XPath format
- A **destination-group** that is composed of destination(s):
  - A **destination** contains the receiver's specifications:
    - IP address
    - port
    - encoding format
    - protocol
- A **subscription** that brings together a sensor-group, a destination-group and a time interval [ms]

In this case, we subscribe to real-time updates of the operational state of all interfaces (`Cisco-IOS-XR-ifmgr-oper`). We expect to receive data regularly, every 10 seconds.

<mark>TASK 4</mark>

Define a data filter for the NETCONF RPC request. This filter is written in XML format and is used to restrict the retrieval of running-configuration: here, we ask only for the model-driven telemetry configuration.

In [ ]:
filter_payload = """
<filter>
 <telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">
 </telemetry-model-driven>
</filter>
"""
netconf_connection.get_config(source='running', filter=filter_payload)

<mark>NOTE</mark> - Observe the content of `<data>` from the `<rpc-reply>`. This is an example of an instantiation of the YANG data model `Cisco-IOS-XR-telemetry-model-driven-cfg`.

### Retrieval of the operational state of model-driven telemetry streaming

Check that MDT streaming is in an active state. Notice that several collections have been made so far:

In [ ]:
filter_payload = """
<filter>
 <telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-oper">
 </telemetry-model-driven>
</filter>
"""
netconf_connection.get(filter=filter_payload)

### Monitoring the streamed model-driven telemetry data

The host 198.18.134.50 already has the collection components for ingesting, storing and visualizing telemetry data. We are using the following set of Docker containers:
- Telegraf - a collection container that runs the cisco_telemetry_mdt plugin for ingestion of model-driven telemetry data
- InfluxDB - a storage container that runs a database which stores the data
- Chronograf - a visualization container that runs a web application which allows the exploration of data

The collector has been configured to listen on port `57777` for incoming connections. IOS XR will initiate the connection, dialling out to the collector. After the connection establishment, the device will begin to stream data to the collector which will send it to a database named `telemetry`.

 <mark>NOTE</mark> - Please refer to the following references for details about the three components and the ingestion of telemetry data:
 - [InfluxData](https://www.influxdata.com/)
 - [Input plugin for Model-driven telemetry](https://github.com/influxdata/telegraf/tree/master/plugins/inputs/cisco_telemetry_mdt)

<mark>TASK 5</mark>

In a separate browser tab, open the web application for data visualization located at http://198.18.134.50:8888. Then, hover over the left hand-side and click on `Dashboards`. Open the `Traffic Monitoring` dashboard.

<mark>NOTE</mark> - If you are missing the dashboard, use the `Import Dashboard` button to upload the dashboard located at `monitoring/chronograf/Traffic Monitoring.json`.


We intend to monitor the input and output data rate for the `GigabitEthernet0/0/0/0`. Currently, there is no data about its traffic.

Notice that we are also monitoring the state of two interfaces, and receiving data in real-time:
- `GigabitEthernet0/0/0/0` - up
- `Loopback0` - down

<img src="images/traffic 0.png">

### Configuration modification for model-driven telemetry streaming

In this section you will add a new sensor-path to the active telemetry subscription. This should send statistics of the interfaces of IOS XR. Use the following sensor-group and sensor-path:

- sensor-group: `SGroupGeneric1`
  - sensor-path: `Cisco-IOS-XR-infra-statsd-oper:infra-statistics/interfaces/interface/latest/generic-counters`

<mark>TASK 6</mark>

`Lock` the `candidate` datastore of the device. This prevents another entity or NETCONF session from making modifications to the datastore. The lock lasts until the `unlock` RPC is sent or until the NETCONF session closes.

In [ ]:
netconf_connection.lock('candidate')

<mark>TASK 7</mark>

Define the configuration for model-driven telemetry. The configuration is in XML format, hierarchically structured, as defined by the YANG data model `Cisco-IOS-XR-telemetry-model-driven-cfg`.

In [ ]:
config = """
<config>

  <telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">

   <sensor-groups>
    <sensor-group>
     <sensor-group-identifier>SGroupGeneric1</sensor-group-identifier>

     <sensor-paths>
      <sensor-path>
       <telemetry-sensor-path>Cisco-IOS-XR-infra-statsd-oper:infra-statistics/interfaces/interface/latest/generic-counters</telemetry-sensor-path>
      </sensor-path>
     </sensor-paths>

    </sensor-group>
   </sensor-groups>

  </telemetry-model-driven>

</config>
"""

netconf_connection.edit_config(target='candidate', config=config, default_operation='merge')

<mark>TASK 8</mark>

Check that the configuration is syntactically and semantically correct:

In [ ]:
netconf_connection.validate('candidate')

<mark>TASK 9</mark>

Commit the changes:

In [ ]:
netconf_connection.commit()

<mark>TASK 10</mark>

Finally, unlock the `candidate` datastore:

In [ ]:
netconf_connection.unlock('candidate')

<mark>TASK 11</mark>

Open the `Traffic Monitoring` dashboard (http://198.18.134.50:8888/sources/0/dashboards/1?lower=now()-5m).

Roughly 10 seconds after the configuration, the receiver will be actively monitoring the input and output data rate for the `GigabitEthernet0/0/0/0`. Currently, there is little traffic, with approximately 1 Kbits/sec input and 7 Kbits/sec output rate.

<img src="images/traffic 1.png">

### Traffic generation

The IOS XR hosts an Iperf server for traffic generation. An Iperf client will contact the server and start sending and receiving information at a 1 Mbits/sec rate. In order to send and receive traffic the following steps are required:

**1. Bring up the Loopback 0**


<mark>TASK 12</mark>

Lock the `candidate` datastore:

In [ ]:
netconf_connection.lock('candidate')

<mark>TASK 13</mark>

Bring up the `Loopback0` by submitting to the `candidate` datastore a configuration change equivalent to the CLI command `no shutdown`:

```
[...]
     <shutdown xc:operation="delete"></shutdown>
[...]
```

In [ ]:
config = """
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>

    <active>act</active>
    <interface-name>Loopback0</interface-name>
    
    <shutdown xc:operation="delete"></shutdown>
    
   </interface-configuration>
  </interface-configurations>
</config>
"""

netconf_connection.edit_config(target='candidate', config=config)

<mark>TASK 14</mark>

Validate the modification of the configuration:

In [ ]:
netconf_connection.validate('candidate')

<mark>TASK 15</mark>

Commit this change to the `running` datastore:

In [ ]:
netconf_connection.commit()

<mark>TASK 16</mark>

Unlock the `candidate` datastore:

In [ ]:
netconf_connection.unlock('candidate')

**2. Starti the Iperf server on the IOS XR device**

<mark>TASK 17</mark>

Connect to the IOS XR's CLI over SSH and start the Iperf server
```
$ ssh cisco@198.18.134.72

RP/0/RP0/CPU0:IOSXR# bash -c iperf -sD
[...]
Running Iperf Server as a daemon
The Iperf daemon process ID : 1266

```

By default, the server will listen on port `5001` for incoming connections.

**3. Start the Iperf client on the VM**

<mark>TASK 18</mark>

Connect to the VM over SSH and start the Iperf client. The client will connect to the host `1.1.1.1` and ask for a bandwidth rate for send and receive of `1000000 bits/sec` (`1 Mbits/sec`).

```
$ ssh cisco@198.18.134.50

[cisco@centos ~]$ iperf -c 1.1.1.1 -b 1000000 -d -t 3600 -i 10
------------------------------------------------------------
Server listening on TCP port 5001
TCP window size: 85.3 KByte (default)
------------------------------------------------------------
------------------------------------------------------------
Client connecting to 1.1.1.1, TCP port 5001
TCP window size: 45.0 KByte (default)
------------------------------------------------------------
[  5] local 198.18.134.50 port 5001 connected with 1.1.1.1 port 64883
recvack failed: Resource temporarily unavailable
[  4] local 198.18.134.50 port 54318 connected with 1.1.1.1 port 5001 (server version is old)
[ ID] Interval       Transfer     Bandwidth
[  5]  0.0-10.0 sec  1.19 MBytes  1.00 Mbits/sec
[  4]  0.0-10.0 sec  1.12 MBytes   944 Kbits/sec
[...]
```

### Real-time monitoring of the generated traffic

<mark>TASK 19</mark>

Return to the `Traffic Monitoring` dashboard (http://198.18.134.50:8888/sources/0/dashboards/1?lower=now()-5m) and confirm the following:

- `Loopback0` is up
- The input and output rates of IOS-XR's `GigabitEthernet0/0/0/0` interface grew to approximately 1 Mbits/sec

<img src="images/traffic 2.png">

### Modifying configuration with confirmed-commit

As seen in the advertised capabilities, IOS-XR supports the `confirmed-commit` capability. This ensures that the changes that are to be committed will be reverted in case a timeout expires without receiving a confirming commit.

This feature is useful in scenarios when the changes have been submitted to the device but the session has closed prematurely, resulting in an inconsistency between the changes expected by the NETCONF client and the state of the device. By providing a timeout interval and requesting a confirmation, the NETCONF client asks the IOS XR device to expect its acknowledgement of applied changes before making them persistent.

To exercise this feature, perform a confirmed-commit that will timeout in 60 seconds:

<mark>TASK 20</mark>

Lock the `candidate` datastore:

In [ ]:
netconf_connection.lock('candidate')

<mark>TASK 21</mark>

Shutdown the `Loopback0`:

In [ ]:
config = """
<config xmlns:xc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interface-configurations xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ifmgr-cfg">
   <interface-configuration>

    <active>act</active>
    <interface-name>Loopback0</interface-name>
    
    <shutdown></shutdown>
    
   </interface-configuration>
  </interface-configurations>
</config>
"""

netconf_connection.edit_config(target='candidate', config=config)

<mark>TASK 22</mark>

Validate the shutdown configuration:

In [ ]:
netconf_connection.validate('candidate')

<mark>TASK 23</mark>

Submit the changes from the `candidate` to the `running` datastore but mention that this is a confirmed commit that times out in 60 seconds:

In [ ]:
netconf_connection.commit(confirmed=True, timeout=u'60')

<mark>TASK 24</mark>

Go back to the `Traffic Monitoring` dashboard. Notice that the device stops receiving traffic once the `Loopback0` is shutdown:

<img src="images/traffic 3.png">

Because this was a commit requesting to be followed by a confirming commit, after approximately 70 seconds, the `Loopback0` reverts its shutdown and comes back up:

<img src="images/traffic 4.png">

<mark>TASK 25</mark>

Unlock the `candidate` datastore:

In [ ]:
netconf_connection.unlock('candidate')

<mark>TASK 26</mark>

Finally, end this laboratory by closing your NETCONF session.

In [ ]:
netconf_connection.close_session()